### NumPy Universal Functions

Universal functions (**ufunc**) zijn functies die element per element worden uitgevoerd op een array - en vectorisatie betekent eenvoudigweg dat de loop en de bewerking worden uitgevoerd in het onderliggend C niveau, en niet in Python.

We hebben al enkele universele functies gezien, zoals de logische functies.  We hebben ook als andere universele functies gebruikt, die eigenlijk als operatoren zijn 'vermomd'.

In [1]:
import numpy as np

In [2]:
arr_1 = np.array([1, 2, 3, 4, 5])
arr_2 = np.array([5, 4, 3, 2, 1])

In [3]:
arr_1 + arr_2

array([6, 6, 6, 6, 6])

Deze `+` operator gebruikt de universal function `np.add`:

In [4]:
np.add(arr_1, arr_2)

array([6, 6, 6, 6, 6])

Dit is ook met andere wiskundige operatoren het geval:

In [5]:
np.multiply(arr_1, arr_2)

array([5, 8, 9, 8, 5])

In [6]:
np.subtract(arr_1, arr_2)

array([-4, -2,  0,  2,  4])

Dit geldt zelfs voor de floor division (`//`), mod (`%`) en exponent (`**`) operatoren:

In [7]:
arr_1 // arr_2

array([0, 0, 1, 2, 5])

In [8]:
np.floor_divide(arr_1, arr_2)

array([0, 0, 1, 2, 5])

In [9]:
arr_1 % arr_2

array([1, 2, 0, 0, 0])

In [10]:
np.mod(arr_1, arr_2)

array([1, 2, 0, 0, 0])

In [11]:
arr_1 ** arr_2

array([ 1, 16, 27, 16,  5])

In [12]:
np.power(arr_1, arr_2)

array([ 1, 16, 27, 16,  5])

Deze universal functions werken ook tussen een array en een scalar (zoals een `int`, `float`, etc), niet alleen tussen twee arrays:

In [13]:
arr_1

array([1, 2, 3, 4, 5])

In [14]:
arr_1 * 2

array([ 2,  4,  6,  8, 10])

In [15]:
arr_1 ** 2

array([ 1,  4,  9, 16, 25])

Dit zijn slechts enkele van de vele universal functions die beschikbaar zijn in NumPy...

We hebben bijvoorbeeld ook de trig functies:

In [16]:
arr = np.linspace(-2 * np.pi, 2 * np.pi, 10)
arr

array([-6.28318531, -4.88692191, -3.4906585 , -2.0943951 , -0.6981317 ,
        0.6981317 ,  2.0943951 ,  3.4906585 ,  4.88692191,  6.28318531])

In [17]:
np.sin(arr)

array([ 2.44929360e-16,  9.84807753e-01,  3.42020143e-01, -8.66025404e-01,
       -6.42787610e-01,  6.42787610e-01,  8.66025404e-01, -3.42020143e-01,
       -9.84807753e-01, -2.44929360e-16])

In [18]:
np.cos(arr)

array([ 1.        ,  0.17364818, -0.93969262, -0.5       ,  0.76604444,
        0.76604444, -0.5       , -0.93969262,  0.17364818,  1.        ])

Via deze link kan je nog meer universal functions ontdekken:

https://numpy.org/doc/stable/reference/ufuncs.html

#### Overwegingen inzake performantie

Herinner je dat we bij de intro inzake ufuncs en vectorisatie als voordeel de enorme toename in snelheid van berekeningen tegenover de standaard Python loop en functies aanhaalden.  Laten we nu even een stukje code bekijken om een idee te krijgen hoeveel beter de performantie van NumPy tov Python is.  Als eerste voorbeeld berekenen we 1/x voor elk element in een list (Python) en een array (NumPy)

In [19]:
# aanmaken van de list
l = []
for x in range(1,10_000_000):
    l.append(x)
l[:10]

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

We starten met een berekening via Python:

In [20]:
from time import perf_counter

start = perf_counter()
new_list = []
for el in l:
    new_list.append(1 / el)
end = perf_counter()
print('Tijd:', end - start)

Tijd: 0.8713662820009631


Nu doen we hetzelfde via een list comprehension:

In [21]:
start = perf_counter()
new_list = [1 / el for el in l]
end = perf_counter()
print('Tijd:', end - start)

Tijd: 0.48504462800337933


De list comprehension is sneller dan de for loop.  We gebruiken nog steeds Python lists en een Python loop!

Laten we even testen met een NumPy array en een vectorized ufunc:

In [22]:
np_l = np.array(l)

In [23]:
np_l.dtype

dtype('int64')

In [24]:
start = perf_counter()
new_arr = 1 / np_l
end = perf_counter()
print('Elapsed:', end - start)

Elapsed: 0.028381673000694718


Zoals je kan zien is er een zeer groot tijdverschil!

We testen dit nu even op een meer gecompliceerd voorbeeld.  Veronderstel dat we een matrix met Open/High/Low/Close data hebben voor een aandeel over een bepaalde periode.  Voor de eenvoud maken we dummy data, en laten we de datum weg.  We willen een matrix met `10_000_000` rijen en `4` kolommen (OHLC), en we gebruiken random nummers tussen `55` en `100`:

Eerst via Python:

In [25]:
import random

In [26]:
num_rows = 10_000_000

random.seed(0)

start = perf_counter()
data = [
    [
        random.randint(70, 85),
        random.randint(85, 100),
        random.randint(55, 70),
        random.randint(70, 85)
    ]
    for _ in range(num_rows)
]
end = perf_counter()
print(data[:2])
print('Elapsed', end - start)

[[82, 98, 56, 78], [85, 97, 64, 85]]
Elapsed 30.92474341599882


Nu maken we een nieuwe lijst die de dagelijkse procentuele variatie (afgerond tot int) van de prijs weergeeft voor elke rij: 

Voor elke rij berekenen we dus:
```
round((high - low) / close * 100)
```

In [27]:
start = perf_counter()
var = [ 
    round((row[1] - row[2]) / row[3] * 100)
    for row in data
]
print(var[:5])
end = perf_counter()
print('Elapsed:', end - start)

[54, 39, 41, 34, 39]
Elapsed: 1.874384121998446


Nu doen we hetzelfde via NumPy:

We maken een NumPy array op basis van onze bestaande Python list:

In [28]:
start = perf_counter()
data_np = np.array(data)
end=perf_counter()
print(data_np[:2])
print('Elapsed:', end - start)

[[82 98 56 78]
 [85 97 64 85]]
Elapsed: 2.8153451769976527


En nu voeren we de berekeningen uit:

In [29]:
start = perf_counter()
var = np.round((data_np[:, 1] - data_np[:, 2]) / data_np[:, 3] * 100)
end = perf_counter()
print(var[:5])
print('Elapsed:', end - start)

[54. 39. 41. 34. 39.]
Elapsed: 0.13548060699395137


Een serieuze verbetering van de performantie!

We kozen ervoor om de NumPy array te maken op basis van de Python list, maar we konden dit ook direct hebben gedaan met NumPy.

Om de NumPy array rechtstreeks te maken, maken we afzonderlijke arrays aan voor de OHLC waarden, en stacken deze horizontaal (`hstack`) om alle kolommen in een enkele 2-D array te combineren: 

In [30]:
np.random.seed(0)
start = perf_counter()
data_np = np.hstack(
    [
        np.random.randint(70, 85, (num_rows, 1)),
        np.random.randint(85, 100, (num_rows, 1)),
        np.random.randint(55, 70, (num_rows, 1)),
        np.random.randint(70, 85, (num_rows, 1))
    ]        
)
end = perf_counter()
print(data_np[:5])
print('Elapsed:', end - start)

[[82 90 57 72]
 [75 89 64 72]
 [70 93 60 70]
 [73 97 69 83]
 [81 96 65 79]]
Elapsed: 0.50022591099696


Dat ging een pak sneller dan het genereren van dezelfde random integers met behulp van Python!

Als we dit allemaal combineren om de performantie van Python en NumPy te vergelijken: 

Python:

In [31]:
num_rows = 10_000_000

In [32]:
random.seed(0)
start = perf_counter()
data = [
    [
        random.randint(120, 180),
        random.randint(180, 200),
        random.randint(100, 120),
        random.randint(120, 180)
    ]
    for _ in range(num_rows)
]
var = [ 
    round((row[1] - row[2]) / row[3] * 100)
    for row in data
]
end = perf_counter()
print('Python Elapsed:', end - start)

Python Elapsed: 29.989695846998075


En nu via NumPy:

In [33]:
np.random.seed(0)
start = perf_counter()
data_np = np.hstack(
    [
        np.random.randint(120, 180, (num_rows, 1)),
        np.random.randint(180, 200, (num_rows, 1)),
        np.random.randint(100, 120, (num_rows, 1)),
        np.random.randint(120, 180, (num_rows, 1))
    ]
)
var = np.round((data_np[:, 1] - data_np[:, 2]) / data_np[:, 3] * 100)
end = perf_counter()
print('NumPy Elapsed:', end - start)

NumPy Elapsed: 0.7654488959960872


Zoals je kan zien is NumPy veel sneller dan Python.

#### Broadcasting

Deze universele functies kunnen we ook met arrays gebruiken die niet dezelfde shape hebben dankzij een techniek die we kennen als **broadcasting*.

https://numpy.org/doc/stable/user/basics.broadcasting.html

We bestuderen broadcasting hier niet, maar enkele eenvoudige gevallen zijn eenvoudig te begrijpen:

Veronderstel dat we een array hebben met daarin getallen die het aantal verkochte eenheden per item weergeven.   Elke rij vertegenwoordigt een order, en elke kolom een specifiek item:

In [34]:
sales = np.array(
    [
        [10, 0, 5, 3],
        [0, 0, 0, 10],
        [1, 1, 0, 0],
        [3, 0, 4, 5]
    ]
)

De volgende array geeft de verkoopprijs voor elk item weer (in dezelfde volgorde als het aantal verkochte eenheden):

In [35]:
unit_price = np.array([100, 50, 20, 10])

Deze array vertegenwoordigt de kost van elk artikel:

In [36]:
unit_cost = np.array([80, 10, 5, 1])

We willen nu de totaal door de verkopen gegenereerde marge berekenen.  Door middel van broadcasting kunnen we de totale verkoopprijs voor elk artikel in elk order berekenen:

In [37]:
sales * unit_price

array([[1000,    0,  100,   30],
       [   0,    0,    0,  100],
       [ 100,   50,    0,    0],
       [ 300,    0,   80,   50]])

Zoals je kan zie werd de `unit_price` ge-**broadcast** voor elke rij, en kon zo de vermenigvuldiging gebeuren.

In [38]:
total_sales = sales * unit_price

We kunnen hetzelfde doen met de kosten:

In [39]:
total_cost = sales * unit_cost
total_cost

array([[800,   0,  25,   3],
       [  0,   0,   0,  10],
       [ 80,  10,   0,   0],
       [240,   0,  20,   5]])

En nu kunnen we het resultaat berekenen:

In [42]:
order_net = total_sales - total_cost
order_net

array([[200,   0,  75,  27],
       [  0,   0,   0,  90],
       [ 20,  40,   0,   0],
       [ 60,   0,  60,  45]])

Om de som van al deze elementen te berekenen zouden we enkele loops kunnen gebruiken, maar NumPy heeft een betere manier om dit te doen.  De `sum` functie die, zelfs in een multi-dimensionale array, alle elementen optelt.

In [43]:
np.sum(order_net)

617

NumPy bevat nog veel meer functionaliteit - meer dan we in de notebooks hebben behandeld.  Je hebt nu evenwel een idee hoe NumPy werkt.  Je bent nu in staat om met behulp van de NumPy documentatie functionaliteit zoeken die je nodig hebt voor specifieke toepassingen.

Eén van de redenen om NumPy te leren, is omdat andere libraries zoals Pandas erop gebouwd zijn.  Pandas biedt een pak functionaliteit die nu is gefocused op datasets. 